### Import plotly and set credentials

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import plotly 
#setting credentials once is enough
#plotly.tools.set_credentials_file(username='swinging-sticks', api_key='9QFKtwuiThwSDeWoiiqG', 
                                 #stream_ids = ['n533fw623f', '11abz0z17d', 'dt2m0v8em8', '1eynig0e1q', 'ji10b2ilrx'])

import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

import numpy as np
import csv

import scipy.integrate as integrate
import time

stream_ids = tls.get_credentials_file()['stream_ids']


### Simulating double pendulum
#### initialize variables and define derivative function

In [2]:
from numpy import sin, cos, pi

# Define constants
g = 9.8      # acceleration due to gravity m/s^2
L1 = 1.0     # length of pendulum 1 in m
L2 = 1.0     # length of pendulum 2 in m
M1 = 1.0     # mass of pendulum 1 in kg
M2 = 1.0     # mass of pendulum 2 in kg
rad = pi/180

# Set initial conditions
th1 = 90    # angle of pendulum 1 in degrees
th2 = 90    # angle of pendulum 2 in degrees
w1 = 0.0     # ang. mom. in degrees/sec
w2 = 0.0     # ang. mom. in degrees/sec
state = np.array([th1, w1, th2, w2])*rad

#Time derivatives
def derivs(state, t):
    dxdt = np.zeros(len(state))   #init derivative
    del_ = state[2]-state[0]     #diff in angle
    
    dxdt[0] = state[1]           #derv. of angle 1
    
    den1 = (M1+M2)*L1 - M2*L1*cos(del_)*cos(del_)
    dxdt[1] = (M2*L1*state[1]*sin(del_)*cos(del_) 
              + M2*g*sin(state[0])*cos(del_)
              + M2*L2*state[3]*state[3]*sin(del_)
              -(M1+M2)*g*sin(state[0]))/den1
    
    dxdt[2] = state[3]
    
    den2 = (L2/L1)*den1
    dxdt[3] = (-M2*L2*state[3]*state[3]*sin(del_)*cos(del_)
               + (M1+M2)*g*sin(state[0])*cos(del_)
               - (M1+M2)*L1*state[1]*state[1]*sin(del_)
               - (M1+M2)*g*sin(state[2]))/den2
    
    return dxdt

### Setting up the plot
#### From plotly - plots the two pendulum positions and the path of the second pendulum 

In [3]:
# drawing the two pendulums

trace1 = Scatter(
    x = [],
    y = [],
    mode = 'lines+markers',    #this shows the actual position of pendulum with a line in-betw
    marker = Marker(size = 12),
    stream = Stream(token = stream_ids[1])
)

# marking the end position movement of second pendulum

trace2 = Scatter(
    x = [],
    y = [],
    mode = 'lines',
    stream = Stream(token = stream_ids[2], maxpoints = 100)
)

# Make data object made up of the 2 scatter objs
data = Data([trace1, trace2])

#### The following will create 2 plots.
####  1) A delta function when there is a flip 2) Number of flips/minute against a guess value

In [4]:
# Plotting when flip happens as a function of time
trace0 = Scatter(
    x = [],
    y = [],
    mode = 'lines',
    stream = Stream(token = stream_ids[0], maxpoints = 1000)
)

data0 = Data([trace0])

# Plotting no of flips/minute against guess value
trace3 = Scatter(
    x = [],
    y = [],
    mode = 'lines',
    stream = Stream(token = stream_ids[3], maxpoints = 10)
)

trace_guess = Scatter(
    x = [],
    y = [],
    mode = 'lines',
    stream = Stream(token = stream_ids[4], maxpoints = 10)
)
data3 = Data([trace3, trace_guess])


#### Setting up axis style and stream objects
#### This step will open the online plots in tabs

In [5]:
axis_style = dict(
    showgrid = False,
    showline = False,
    zeroline = False
)

layout = Layout(
    title='Double Pendulum Simulation',  # set plot's title
    xaxis=XAxis(
        axis_style,     # add style options
        range=[-2,2]    # set x-axis range
    ),
    yaxis=YAxis(
        axis_style,     # add style options
        range=[-2,2]  # set y-axis range
    ),
    showlegend=False    # remove legend
)


# Make figure object
fig = Figure(data=data, layout=layout) # plot of pendulum
fig0 = Figure(data=data0) # plot of flips per minute
fig3 = Figure(data=data3) # plot of number of flips/minute vs guess value

# (@) Send fig to Plotly, initialize streaming plot, open tab
unique_url = py.plot(fig, filename='s7_streaming-double-pendulum')
unique_url0 = py.plot(fig0, filename='Flip')
unique_url3 = py.plot(fig3, filename='Flips-per-minute')

# (@) Make instances of the stream link object, 
#     with same stream id as the stream id object in traces
s1 = py.Stream(stream_ids[1]) #fig
s2 = py.Stream(stream_ids[2]) #fig

s0 = py.Stream(stream_ids[0]) #fig0

s3 = py.Stream(stream_ids[3]) #fig3
s4 = py.Stream(stream_ids[4]) #fig3

# (@) Open streams
s1.open()
s2.open()

s0.open()

s3.open()
s4.open()

#### Run Simulation to create 'historical data'
#### SKIP THIS STEP. Takes a long time.. instead read from csv file as shown below

In [ ]:
# Running ODE 1000 times to generate histogram

N = 1000   # number of time integrate.odeint() integrations 
i = 0   # init. counter

flip_total = []
counts = np.zeros(N)

# Solve the system of ODEs N times. Each simulation is for 60s
while i < N:

    dt = 0.1                    
    t = np.arange(0.0, 60, dt)  # sampled at 0.1 second steps (of 600 steps in all) units seconds. 

    # Solve the system of ODEs, for times in t!
    y = integrate.odeint(derivs, state, t)

    x1 = L1*sin(y[:,0])        # convert angles to x-y coordinates
    y1 = -L1*cos(y[:,0])
    x2 = L2*sin(y[:,2]) + x1   #   for both pendulums
    y2 = -L2*cos(y[:,2]) + y1

   
    indices = [ind for ind,a in enumerate(y1) if a>0.9] #Checking where the first pendulum become close to horizontal
    flip = np.zeros(len(t))
    
    for ind in indices:
        try:
            n = np.absolute(np.sign(x1[ind]) - np.sign(x1[ind+1]))  # checking if x changes sign
            if n==2:
                flip[ind] = 1 #flip = 1 when there is a flip. 
                        #flip = 0 when there is no flip  
        except:
            continue
    
    flip_total.append(zip(i*60+t, flip))
    counts[i] = np.sum(flip)
    # Set the new initial state
    state = np.array([y[-1,0], y[-1,1], y[-1,2], y[-1,3]])
    i += 1                      # add to counter
    

#### Write historical data
#### SKIP this as well

In [ ]:
with open('flip_total.csv', 'wb') as csvfile:
    fieldnames = ['time (s)', 'flip']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    for a in flip_total:
        for b in a:
            writer.writerow([b[0], b[1]])

with open('counts.csv', 'wb') as csvfile:
    fieldnames = ['time(m)', 'flip/min']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    for a, b in enumerate(counts):
        writer.writerow([a,b])

#### Read historical data

In [20]:
counts = []
with open('counts.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        counts.append(float(row['flip/min']))
                      
h = np.histogram(counts, range(int(max(counts))))

#### Guess from historical data

In [21]:
def guess(h): 
    cum = np.cumsum(h[0])
    g = np.random.randint(1000)
    ind = [i for i,a in enumerate(cum) if a>g]
    return(ind[0])

#### Run simulation 
#### This writes data into the plots

In [22]:
N = 10   # number of time integrate.odeint() integrations 
i = 0   # init. counter
# Delay start of stream by 5 sec (time to switch tabs)
time.sleep(5)

# (-) Solve the system of ODEs N times
while i < N:

    dt = 0.1                   # create a time array from 0..100  
    t = np.arange(0.0, 60, dt)  # sampled at 0.05 second steps (of 400 steps in all)

    # Solve the system of ODEs, for times in t!
    y = integrate.odeint(derivs, state, t)

    x1 = L1*sin(y[:,0])        # convert angles to x-y coordinates
    y1 = -L1*cos(y[:,0])
    x2 = L2*sin(y[:,2]) + x1   #   for both pendulums
    y2 = -L2*cos(y[:,2]) + y1

    # (!) Write the solutions to Plotly's servers, 
    #     1 per stream, 1 point at a time
    for (x1i, y1i, x2i, y2i) in zip(x1, y1, x2, y2):

        # (@) Write list corresponding to 3 pendulum nodes,
        #     overwriting the data on the plot
        s1.write(dict(x=[0, x1i, x2i], y=[0, y1i, y2i]))

        # (@) Write 1 point corresponding to 1 pt of path,
        #     appending the data on the plot
        s2.write(dict(x=x2i, y=y2i))

        time.sleep(0.08)  # (!) plot pts 80 ms at a time, for smoother plotting

    indices = [ind for ind,a in enumerate(y1) if a>0.9] #Checking where the first pendulum become close to horizontal
    flip = np.zeros(len(t))
    
    for ind in indices:
        try:
            n = np.absolute(np.sign(x1[ind]) - np.sign(x1[ind+1]))  # checking if x changes sign
            if n==2:
                flip[ind] = 1 #flip = 1 when there is a flip. 
                        #flip = 0 when there is no flip  
        except:
            continue
   
    s0.write(dict(x = i*60+t, y = flip))
    
    s3.write(dict(x = i, y = np.sum(flip)))
    s4.write(dict(x = i, y = guess(h)))  #h == histogram of counts, calculated from historical data

    # Set the new initial state
    state = np.array([y[-1,0], y[-1,1], y[-1,2], y[-1,3]])
    i += 1                      # add to counter
    
    
# (@) Close both streams when done plotting
s1.close()
s2.close()

s0.close()

s3.close()
s4.close()

### Testing area